# Forecasting the 2022 Election: A PCS Workshop

### Created by Ian Joffe, Spring 2022

In [ ]:
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
%store -r solutions

## Playing with the Beta Distribution

In [ ]:
alpha = 1
beta = 1

x = np.linspace(0,1,200)
y = stats.beta.pdf(x, alpha, beta)
plt.plot(x, y);
plt.title("Beta(" + str(alpha) + ", " + str(beta) + ")");

### Conceptual Q1

What is the affect of increasing $\alpha$? What is the affect of increasing $\beta$? What happens if we set $\alpha = \beta$ and increase both? For $\alpha, \beta > 1$, approximately where is the distribution centered?

## Gathering Data

In [ ]:
# The following cells must be run in order, once

In [ ]:
# We use fivethirtyeight's generic ballot polling average for our forecast
polls = pd.read_csv("generic_topline_historical.csv")[["modeldate", "dem_estimate", "rep_estimate"]]
polls["dem_lead"] = polls["dem_estimate"] - polls["rep_estimate"]
polls = polls.drop(["dem_estimate", "rep_estimate"], axis=1)
polls = polls[polls["modeldate"].str.slice(0,1) == "4"]
print("All April Polls:")
polls.head()

In [ ]:
polls["year"] = polls["modeldate"].str.slice(start=-4).astype(int)
polls = polls[np.mod(polls["year"], 2) == 0]
polls = polls[polls["year"] % 2 == 0]
polls = polls.groupby("year").mean()
# I have to manually add data from 2018 and 2020 because it's from a different data set
additional_data = pd.DataFrame({"Year": [2018, 2020], "dem_lead": [7.1093803, 7.749738]})
additional_data = additional_data.set_index("Year")
polls = polls.append(additional_data)
# Append the final voting margins for each year to the data set
final_margins = [0, -1, 0, -5, -3, 8, 11, -7, 1, -6, 1, 9, 3]
polls["true_result"] = final_margins
polls["dem_lead"] = polls["dem_lead"] / 100
polls["true_result"] = polls["true_result"] / 100
print("April Polling Averages from Midterm Years")
polls

In [ ]:
polls_abs = polls.copy(deep=True)
polls_abs["dem_lead"] = .5 + polls["dem_lead"] / 2
polls_abs["true_result"] = .5 + polls["true_result"] / 2
polls_abs = polls_abs.rename(columns={"dem_lead": "dem_pct"})

## Implementing Gradient Descent

In [ ]:
def beta_reparam(x, mode, concentration):
    return stats.beta.pdf(x, mode * (concentration - 2) + 1, (1 - mode) * (concentration - 2) + 1)

In [ ]:
mode = .5
concentration = 300

x = np.linspace(0,1,200)
y = beta_reparam(x, mode, concentration)
plt.plot(x, y);
plt.title("Beta(mode=" + str(mode) + ", concentration=" + str(concentration) + ")");

### Conceptual Q2

Like before, experiment with the affects of changing the mode and concentration parameters. 

### Code Q1

```calculate_result_probabilities``` returns a list of the probabilities of the true outcome for each election since 1996, with respect to their polling leads and a given confidence in polling. 

In [ ]:
def calculate_result_probabilities(confidence):
    true_results = polls_abs["true_result"].values
    leads = polls_abs["dem_pct"].values
    n = len(true_results)
    return np.array([beta_reparam(...[i], ...[i], ...) for i in range(n)])

In [ ]:
# Run to view solution
print(solutions["calculate_result_probabilities"])

### Code Q2

Say we choose some value for confidence. Design an appropriate loss function that calculates the loss for the chosen confidence over elections since 1996. Your loss function should call ```calculate_result_probabilities(confidence)```

In [ ]:
def loss(confidence):
    return ...

Plot the loss function for many different confidence values. 

In [ ]:
confs = np.linspace(0, 2000, 500)
losses = []
for c in confs:
    losses.append(loss(c))
plt.plot(confs, losses);
plt.xlabel("Confidence Parameter Value");
plt.ylabel("Loss");

In [ ]:
# Run to view solution
print(solutions["loss"])

In [ ]:
# Use the method of finite differences to calculate the derivative of our loss function at a given confidence

def finite_difference(func, x, epsilon=.0001):
    return (func(x+epsilon) - func(x-epsilon)) / (2 * epsilon)

def loss_deriv_fd(confidence, epsilon=.001):
    return finite_difference(loss, confidence) 

### Code Q3

Implement the Gradient Descent algorithm. Line 1 randomly initializes our initial confidence. Line 2 sets the number of "epochs," which is the number of times we go over all our data, calculate a new loss, and reassign ```optimal_confidence```. Line 3 sets the learning rate - you can try messing with this, but it's preset to a good value. Write one line to replace the elipses which updates ```optimal_confidence``` according to the gradient descent formula. 

In [ ]:
initial_confidence = np.random.uniform(1, 100)
epochs = 500
learning_rate = 10

optimal_confidence = initial_confidence
for epoch in range(epochs):
    ...
    
print(optimal_confidence)

In [ ]:
print(solutions["gradient descent"])

## Implementing Monte Carlo

In [ ]:
# In this section, we use FiveThiryEight's metrics for paritsan lean and elasticity.
# The partisan lean metric is on the correct scale for our purpose, but the elasticity metric is normalized around 1. 
# This function takes in a FiveThirtyEight elasticity value, and applies some transformation to it to make it more useful in 
# our context. 
def elasticity_to_confidence(elasticity):
    return 500 - 1000 * (elasticity - 1)

In [ ]:
# The current percent of the vote going to democrats, adapted from FiveThirtyEight's 2022 generic ballot tracker, 
# as of this morning (4/17). Recall we assume that all votes go to dem or rep, so dem_pct = 0.5 + dem_lead / 2
current_dem_pct = .4885

In [ ]:
# Import the FiveThirtyEight's data about each district. 
# Here, we apply another simplifcation. Since not all district maps have been approved, we assume the 2022 house districts
# and their partisan leans match those of 2020. 
districts = pd.read_csv("partisan_leans.csv", header=None).loc[:,[0,1,2]].rename(columns={0:"district", 1:"lean", 2:"elasticity"})
print("The FiveThirtyEight partisan lean and elasticity of each district:")
districts.head()

In [ ]:
# Change the scale of the lean metric to match our standard
districts["lean"] = districts["lean"].str.slice(2).astype(float) * np.where(districts["lean"].str.slice(stop=1) == "D", 1, -1) / 100

### Code Q4

Set the values of ```mode``` and ```concentration``` for the simulation of a singular district under the beta distribution. 

In [ ]:
def simulate_district(natl_gen_ballot, state_lean, elasticity):
    mode = ...
    concentration = ...
    if mode > 1:
        return 1
    elif mode < 0:
        return 0
    else: 
        return stats.beta.rvs(mode * (concentration - 2) + 1, (1 - mode) * (concentration - 2) + 1)

In [ ]:
print(solutions["simulate_district"])

This is the simulation for the entire country. Note how here, the mode is set to the current generic ballot lead for democrats, and the concentration is the set to the optimal concentration we set earlier. This function calls ```simulate_district``` for each district, keeps a list of the probability of democrats winning each district, and returns the number of districts in which democrats are at least 50% likely to win. 

In [ ]:
def simulate_country():
    mode = current_dem_lead
    concentration = optimal_confidence
    natl_avg = stats.beta.rvs(mode * (concentration - 2) + 1, (1 - mode) * (concentration - 2) + 1)
    dist_outcomes = []
    for district in districts.iterrows():
        dist_outcomes.append(simulate_district(natl_avg, district[1]["lean"], elasticity_to_confidence(district[1]["elasticity"])))
    # Returns the number of seats that democrats "win"
    return sum(np.array(dist_outcomes) > 0.5)

We run simulate_country ```n_simulations``` times, and make a list of the seats that democrats win each of them. In the next cell, we make a histogram of the results. 

In [ ]:
n_simulations = 500
dem_seats = []
for i in range(n_simulations):
    dem_seats.append(simulate_country())
dem_seats = np.array(dem_seats)

In [ ]:
plt.hist(dem_seats, bins=20);
plt.axvline(218, color="red", linewidth=1);
plt.title("Probability of Dem Majority: " + str(sum(dem_seats >= 218) / len(dem_seats)) + "\nMean Dem Seats: " + str(int(np.mean(dem_seats))));
plt.xlabel("Dem Seats");

## A Much More Simple Model

### Code Q5

In an attempt to simplify our model, we say that the house goes whatever way the tipping point district goes. The following code block should execute this strategy rather efficiently.

In [ ]:
# Find the partisan lean of the tipping point district
# Recall the we can get a list of all partisan leans with districts["lean"]
# Perhaps google a function you can apply to this list (or, more accurately, pandas series)?
tipping_point_dist_lean = ...

print("The tipping point district has a lean of " + str(tipping_point_dist_lean))

# Now, we plan to plug this district's mode and concentration into the beta distribution.
# What would be use for the mode?
tipping_point_dist_mode = ...

tipping_point_dist_concentration = optimal_concentration

# Convert back to the alpha/beta paramaterization and plug in to the beta distribution
alpha = tipping_point_dist_mode * (tipping_point_dist_concentration - 2) + 1
beta = (1 - tipping_point_dist_mode) * (tipping_point_dist_concentration - 2) + 1
prob_dem_win = 1 - stats.beta.cdf(.5, alpha, beta)
print("According to the simple model, Democrats have a " + str(prob_dem_win) + " chance of winning the house")

In [ ]:
print(solutions["tipping_point_model"])

## A Slightly More Complex Model

### The "Midterm Penalty"

Add the ```dem_pres``` column to the polls table, which is is 0 in general elections years, 1 if the sitting president is a democrat, and -1 if the sitting president is a republican. 

In [ ]:
polls_abs["dem_pres"] = [0, 1, 0, -1, 0, -1, 0, 1, 0, 1, 0, -1, 0]
polls_abs.head()

### Code Q6

Our setup for gradient descent is very similar to before, but now we have two to optimize (the confidence and the penalty) instead of one (just the confidence) like before. Loss is a function of both parameters, and it has two derivatives: one with respect to each parameter. Like before, we use the finite difference method to calculate each derivative. Complete the two finite difference functions to find each of the two derivatives. You can use a non-general version of the old univariate finite difference function as a model: 

```
def finite_difference(confidence, epsilon=.0001):
    return (loss(confidence+epsilon) - loss(confidence-epsilon)) / (2 * epsilon)
```

In each of the new functions we should be applying the epsilon transformation to one of the two parameters, and keeping the other one constant. 

In [ ]:
def calculate_result_probabilities(confidence, penalty):
    true_results = polls_abs["true_result"].values
    leads = polls_abs["dem_lead"].values
    dem_pres = polls_abs["dem_pres"].values
    n = len(true_results)
    # Take a look at this line
    # Our new mode is now the generic ballot (like you set up in Code Q1) MINUS a penalty, which is applied to
    # niether party in a general election year, and the sitting presidential party in midterm years. 
    return np.array([beta_reparam(true_results[i], leads[i] - penalty*dem_pres[i], confidence) for i in range(n)])

def loss(confidence, penalty):
    return -sum(calculate_result_probabilities(confidence, penalty) ** (1/3))

def loss_deriv_wrt_confidence(confidence, penalty, epsilon=.001):
    def finite_difference(confidence, penalty):
        return (loss(..., ...) - loss(..., ...)) / (2 * epsilon)
    return finite_difference(confidence, penalty)

def loss_deriv_wrt_penalty(confidence, penalty, epsilon=.001):
    def finite_difference(confidence, penalty):
        return (loss(..., ...) - loss(..., ...)) / (2 * epsilon)
    return finite_difference(confidence, penalty)

In [ ]:
print(solutions["multivariate fd"])

Perform gradient descent. This is just a doubled version of the algorithm that you wrote earlier. We initialize *both* parameters, and then apply the update step to *both* parameters, calling each's own derivative. 

In [ ]:
initial_confidence = np.random.uniform(1, 100)
initial_penalty = np.random.uniform(0, .1)
epochs = 500
learning_rate_confidence = 10
learning_rate_penalty = .0001

optimal_confidence = initial_confidence
optimal_penalty = initial_penalty
for epoch in range(epochs):
    optimal_confidence -= learning_rate_confidence * loss_deriv_wrt_confidence(optimal_confidence, optimal_penalty)
    optimal_penalty -= learning_rate_penalty * loss_deriv_wrt_penalty(optimal_confidence, optimal_penalty)
    
print(optimal_confidence)
print(optimal_penalty)

In [ ]:
def simulate_district(natl_gen_ballot, state_lean, elasticity):
    mode = natl_gen_ballot + state_lean
    concentration = elasticity
    if mode > 1:
        return 1
    elif mode < 0:
        return 0
    else: 
        return stats.beta.rvs(mode * (concentration - 2) + 1, (1 - mode) * (concentration - 2) + 1)
    
def simulate_country():
    # The only change from earlier is the following line.
    # The midterm penalty is applied to democrats because Joe Biden is president. 
    mode = current_dem_lead - optimal_penalty
    concentration = optimal_confidence
    natl_avg = stats.beta.rvs(mode * (concentration - 2) + 1, (1 - mode) * (concentration - 2) + 1)
    dist_outcomes = []
    for district in districts.iterrows():
        dist_outcomes.append(simulate_district(natl_avg, district[1]["lean"], elasticity_to_confidence(district[1]["elasticity"])))
    return sum(np.array(dist_outcomes) > 0.5)

Just like before, simulate and graph the election results ```n_simulations``` times. Though you may encounter some randomness, Democrats should generally do worse in this model than earlier due to the penalty. 

In [ ]:
n_simulations = 500
dem_seats = []
for i in range(n_simulations):
    dem_seats.append(simulate_country())
dem_seats = np.array(dem_seats)

plt.hist(dem_seats, bins=20);
plt.axvline(218, color="red", linewidth=1);
plt.title("Probability of Dem Majority: " + str(sum(dem_seats >= 218) / len(dem_seats)) + "\nMean Dem Seats: " + str(int(np.mean(dem_seats))));
plt.xlabel("Dem Seats");

### Covariant Districts

### Conceptual Q3

Some distircts are likely to vote similarly to others, often based on geographic or demographic factors. Improve your model by accounting for this in the ```covariant_districts``` dictionary. Each key district maps to one or multiple value districts, which its outcomes correlate to. Using your knowledge of geography and demography (plus http://proximityone.com/cd.htm, a resource which has demographic info on congrssional districts at a table in the middle, and https://www.govtrack.us/congress/members/map, a map of US congressional districts), improve the model by accounting for covariant districts. I've already added districts from downtown NYC, Los Angeles, and Chicago - although I won't guarantee that they are actually correlated, you should check that!

In [ ]:
covariant_districts = {"CA-34": ["NY-12", "IL-7"], "NY-12": ["CA-34", "IL-7"], "IL-7":["CA-34", "NY-12"]}

The following code is very similar to the functions we wrote earlier - the only difference is it now accounts for the covariant districts. First, we randomize the order of the districts. Then, for each district, if there are no covariants yet we do the same as before, but if there are covariants we apply them. 

The ```elasticity_factor```, which is between 0 and 1, is the factor by which we multiply the elasticity by for each covariant state. It makes sense that elasticity decreases for every state that is covariant. 

The ```average_factor``` is applied so that if a district correlates to one other district, the mode of its beta distribution is the national generic ballot times one minus the factor, plus the covariant state times the factor. 

You're welcome to play with these two parameters. In a much more complex model, each covariant will have its own value for these two factors, and those values could be optimized using a method like gradient descent. 

In [ ]:
covariant_state_elasticity_factor = .75
covariant_state_average_factor = .3

def simulate_district(natl_gen_ballot, district, prev_outcomes):
    
    def get_elasticity():
        elasticity = elasticity_to_confidence(district["elasticity"])
        covariants = covariant_states.get(district["district"][:2])
        if covariants is not None:
            for covariant in covariants:
                if covariant in list(prev_outcomes.keys()):
                    elasticity *= covariant_state_elasticity_factor
        return elasticity
        
    def get_avg():
        avg = natl_gen_ballot + district["lean"]
        covariants = covariant_states.get(district["district"][:2])
        if covariants is not None:
            for covariant in covariants:
                if covariant in list(prev_outcomes.keys()):
                    avg = prev_outcomes[covariant] * convariant_state_average_factor + avg * (1-covariant_state_average_factor)
        return avg
            
    mode = get_avg()
    concentration = get_elasticity()
    if mode > 1:
        return 1
    elif mode < 0:
        return 0
    else: 
        return stats.beta.rvs(mode * (concentration - 2) + 1, (1 - mode) * (concentration - 2) + 1)
    
def simulate_country():
    mode = current_dem_lead
    concentration = optimal_confidence
    natl_avg = stats.beta.rvs(mode * (concentration - 2) + 1, (1 - mode) * (concentration - 2) + 1)
    dist_outcomes = {}
    for district in districts.iterrows():
        dist_outcomes[district[1]["district"]] = simulate_district(natl_avg, district[1], dist_outcomes)
    return sum(np.array(list(dist_outcomes.values())) > 0.5)

Like before, run the simulations. Since we're only applying a few covariances you probably won't notice a big difference in the results. 

In [ ]:
n_simulations = 500
dem_seats = []
for i in range(n_simulations):
    dem_seats.append(simulate_country())
dem_seats = np.array(dem_seats)

plt.hist(dem_seats, bins=20);
plt.axvline(218, color="red", linewidth=1);
plt.title("Probability of Dem Majority: " + str(sum(dem_seats >= 218) / len(dem_seats)) + "\nMean Dem Seats: " + str(int(np.mean(dem_seats))));
plt.xlabel("Dem Seats");

You've now written the key components of a working model for the 2022 house election! We made a lot of assumptions and simplifications that might affect our accuracy, but all in all, our model should actually be pretty decent. Once the industry standard models (e.g. FiveThirtyEight, The Economist) come out, it'll be interesting to see how close their probabilities match up with ours.